In [76]:
import numpy as np 
import pandas as pd
import seaborn as sns
import sklearn 
from sklearn import linear_model

In [58]:
def import_data(file):
    return pd.read_csv(file, header=0)

df = import_data("houses_to_rent_v2.csv")

In [59]:
# What's the shape of our data, how many houses, how many variables? 

print(df.shape)

print(df)

(10692, 13)
                 city  area  rooms  bathroom  parking spaces floor     animal  \
0           São Paulo    70      2         1               1     7      acept   
1           São Paulo   320      4         4               0    20      acept   
2        Porto Alegre    80      1         1               1     6      acept   
3        Porto Alegre    51      2         1               0     2      acept   
4           São Paulo    25      1         1               0     1  not acept   
...               ...   ...    ...       ...             ...   ...        ...   
10687    Porto Alegre    63      2         1               1     5  not acept   
10688       São Paulo   285      4         4               4    17      acept   
10689  Rio de Janeiro    70      3         3               0     8  not acept   
10690  Rio de Janeiro   120      2         2               2     8      acept   
10691       São Paulo    80      2         1               0     -      acept   

           furn

We have 10692 houses, with 13 dimensions. Here we don't have a useless ID or something so we get rid of some 

In [60]:
# It seems in the column floor that some hyphens are used instead of 0s

df['floor'].replace(("-"),(0),inplace=True)

print(df)

                 city  area  rooms  bathroom  parking spaces floor     animal  \
0           São Paulo    70      2         1               1     7      acept   
1           São Paulo   320      4         4               0    20      acept   
2        Porto Alegre    80      1         1               1     6      acept   
3        Porto Alegre    51      2         1               0     2      acept   
4           São Paulo    25      1         1               0     1  not acept   
...               ...   ...    ...       ...             ...   ...        ...   
10687    Porto Alegre    63      2         1               1     5  not acept   
10688       São Paulo   285      4         4               4    17      acept   
10689  Rio de Janeiro    70      3         3               0     8  not acept   
10690  Rio de Janeiro   120      2         2               2     8      acept   
10691       São Paulo    80      2         1               0     0      acept   

           furniture  hoa (

In [61]:
#Here we get a sense of the data swe're dealing with

print(df.head(5))


df.describe()

           city  area  rooms  bathroom  parking spaces floor     animal  \
0     São Paulo    70      2         1               1     7      acept   
1     São Paulo   320      4         4               0    20      acept   
2  Porto Alegre    80      1         1               1     6      acept   
3  Porto Alegre    51      2         1               0     2      acept   
4     São Paulo    25      1         1               0     1  not acept   

       furniture  hoa (R$)  rent amount (R$)  property tax (R$)  \
0      furnished      2065              3300                211   
1  not furnished      1200              4960               1750   
2  not furnished      1000              2800                  0   
3  not furnished       270              1112                 22   
4  not furnished         0               800                 25   

   fire insurance (R$)  total (R$)  
0                   42        5618  
1                   63        7973  
2                   41        3841 

,area,rooms,bathroom,parking spaces,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
count,10692.000000,10692.000000,10692.000000,10692.000000,1.069200e+04,10692.000000,10692.000000,10692.000000,1.069200e+04
mean,149.217920,2.506079,2.236813,1.609147,1.174022e+03,3896.247194,366.704358,53.300879,5.490487e+03
std,537.016942,1.171266,1.407198,1.589521,1.559231e+04,3408.545518,3107.832321,47.768031,1.648473e+04
min,11.000000,1.000000,1.000000,0.000000,0.000000e+00,450.000000,0.000000,3.000000,4.990000e+02
25%,56.000000,2.000000,1.000000,0.000000,1.700000e+02,1530.000000,38.000000,21.000000,2.061750e+03
50%,90.000000,2.000000,2.000000,1.000000,5.600000e+02,2661.000000,125.000000,36.000000,3.581500e+03
75%,182.000000,3.000000,3.000000,2.000000,1.237500e+03,5000.000000,375.000000,68.000000,6.768000e+03
max,46335.000000,13.000000,10.000000,12.000000,1.117000e+06,45000.000000,313700.000000,677.000000,1.120000e+06


In [62]:
# some columsn are text, whilst they should be numeric i.e. animal, let's change that

df['animal'].replace(('acept', 'not acept'), (1, 0), inplace=True)

df['furniture'].replace(('furnished', 'not furnished'), (1,0), inplace=True)


In [63]:
# the city column is an issue, perhaps three different models will be needed
# let's first create a model without the city

dfwo = df.drop("city",axis=1, inplace=False)

dfwo.head(5)

In [67]:
# we turn the entire dataset into integers

dfwo = dfwo.astype(dtype=np.int64)

In [72]:
#we check that it did turn everything into an integer

dfwo.isin(['Incluso']).any()

area                   False
rooms                  False
bathroom               False
parking spaces         False
floor                  False
animal                 False
furniture              False
hoa (R$)               False
rent amount (R$)       False
property tax (R$)      False
fire insurance (R$)    False
total (R$)             False
dtype: bool

In [75]:
# Wwe shuffle the data to make sure that there wasn't a bias whilst inputing data

dfwo = dfwo.sample(frac=1).reset_index(drop=True)



In [89]:
dfwo.describe()

,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
count,10692.000000,10692.000000,10692.000000,10692.000000,10692.000000,10692.000000,10692.000000,1.069200e+04,10692.000000,10692.000000,10692.000000,1.069200e+04
mean,149.217920,2.506079,2.236813,1.609147,5.067995,0.777778,0.243734,1.174022e+03,3896.247194,366.704358,53.300879,5.490487e+03
std,537.016942,1.171266,1.407198,1.589521,6.069050,0.415759,0.429354,1.559231e+04,3408.545518,3107.832321,47.768031,1.648473e+04
min,11.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,450.000000,0.000000,3.000000,4.990000e+02
25%,56.000000,2.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.700000e+02,1530.000000,38.000000,21.000000,2.061750e+03
50%,90.000000,2.000000,2.000000,1.000000,3.000000,1.000000,0.000000,5.600000e+02,2661.000000,125.000000,36.000000,3.581500e+03
75%,182.000000,3.000000,3.000000,2.000000,8.000000,1.000000,0.000000,1.237500e+03,5000.000000,375.000000,68.000000,6.768000e+03
max,46335.000000,13.000000,10.000000,12.000000,301.000000,1.000000,1.000000,1.117000e+06,45000.000000,313700.000000,677.000000,1.120000e+06


In [80]:
# Now to start our multiple regression, we want to 

X = dfwo[['area','rooms','bathroom','parking spaces','floor','animal','furniture','hoa (R$)','rent amount (R$)','property tax (R$)','fire insurance (R$)']]
y = dfwo ['total (R$)']

regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [91]:
# Finally we can predict a house price based on some factors

price = regr.predict([[149.217920,2.506079,2.236813,1.609147,5.067995,0.777778,0.243734,1.174022,3896.247194,366.704358,53.300879]])

print(price)



[4317.52520759]
